### CIFAR10 Classifer Evaluation
In this notebook, we evaluate pre-trained **CNN** to classify images from the CIFAR-10 database.

The images in this database are small color images that fall into one of ten classes; some example images are pictured below.

<img src='notebook_ims/cifar_data.png' width=50% height=50% />

In [1]:
!wget -q https://datasets.mlpack.org/cifar-10_test-1k.csv.gz

In [2]:
!gunzip -d cifar-10_test-1k.csv.gz

gzip: cifar-10_test-1k.csv already exists;	not overwritten


In [3]:
!wget -q https://datasets.mlpack.org/cifarNet.xml

In [4]:
// Import necessary library headers.
#include <mlpack/xeus-cling.hpp>
#define MLPACK_ENABLE_ANN_SERIALIZATION
#include <mlpack.hpp>

In [5]:
using namespace mlpack;

In [6]:
using namespace arma;

In [7]:
using namespace ens;

In [8]:
arma::Row<size_t> getLabels(const arma::mat& yPreds) 
{
    arma::Row<size_t> yLabels(yPreds.n_cols);
    for (arma::uword i = 0; i < yPreds.n_cols; ++i)
    {
        yLabels(i) = yPreds.col(i).index_max();
    }
    return yLabels;
}

### Network Architecture

Our Model using Convolutional Neural Network CNN architecture.
* Convolutional layers - which can be thought of as stack of filtered images.
* Maxpooling layers - which reduce the x-y size of an input, keeping only the most _active_ pixels from the previous layer.
* Linear layer - Applies a linear transformation to the incoming data: $ y=xA^T+b $

```
32 x 32 x 3 --- conv (6 feature maps of kernel size 5 x 5 with stride = 1) ---> 28 x 28 x 6
28 x 28 x 6 ------------------------ Leaky ReLU ------------------------------> 28 x 28 x 6 
28 x 28 x 6 ------- max pooling (kernel size of 2 x 2 with stride = 2) -------> 14 x 14 x 6
14 x 14 x 6 --- conv (16 feature maps of kernel size 5 x 5 and stride = 1) ---> 10 x 10 x 16
10 x 10 x 16 ----------------------- Leaky ReLU ------------------------------> 10 x 10 x 16
10 x 10 x 16 ------ max pooling (kernel size of 2 x 2 with stride = 2) -------> 5 x 5 x 16
5 x 5 x 16  ------------------------- Linear ---------------------------------> 10
```

An example 2 convolutional layers is shown in the image below,

<img src='notebook_ims/2_layer_conv.png' height=50% width=50% />

In [9]:
// Create the Feed Forward Neural Network object, into which our
// pretrained model will be loaded.
//
// It's not necessary to specify the model architecture, since that is
// saved along with the model.  Nonetheless, for edification, we include
// the architecture below (commented out).
FFN<NegativeLogLikelihood, RandomInitialization> model;

// model.Add<Convolution>(6, 5, 5, 1, 1, 0, 0); 
// model.Add<LeakyReLU>(); 
// model.Add<MaxPooling>(2, 2, 2, 2, true);
// model.Add<Convolution>(16, 5, 5, 1, 1, 0, 0);
// model.Add<LeakyReLU>();
// model.Add<MaxPooling>(2, 2, 2, 2, true);
// model.Add<Linear>(120);
// model.Add<LeakyReLU>();
// model.Add<Linear>(84);
// model.Add<LeakyReLU>();
// model.Add<Linear>(10);
// model.Add<LogSoftMax>();

###  Load the Model with the Lowest Validation Loss

In [10]:
mlpack::data::Load("cifarNet.xml", "model", model, true);

[FATAL] XML Parsing failed - provided NVP (outputLayer) not found



Standard Exception: fatal error; see Log::Fatal output

In [12]:
// Matrix for storing test feeature & labels.
mat testData, testY;
// Load the test data.
mlpack::data::Load("./cifar-10_test-1k.csv", testData, true);
// Drop the header column.
testData.shed_col(0);
// Remove labels before predicting.
testY = testData.row(testData.n_rows - 1);
testData.shed_row(testData.n_rows - 1);

In [13]:
cout << "Starting Prediction on testset ..." << endl;
mat testPredProbs;
// Get predictions on test data points.
model.Predict(testData, testPredProbs);
arma::Row<size_t> testPreds = getLabels(testPredProbs);
double testAccuracy = arma::accu(testPreds == testY) / (double) testY.n_elem * 100;

Starting Prediction on testset ...


Standard Exception: FFN::Predict(): cannot use network with no layers!

In [14]:
cout << "Accuracy on testset = " << testAccuracy << "%" << endl;

Accuracy on testset = 0%
